In [1]:
# =============================================================================
# CẤU HÌNH HUẤN LUYỆN
# =============================================================================

NUM_EPOCHS = 100
LEARNING_RATE = 1e-4
WEIGHT_DECAY = 1e-4
BATCH_SIZE=16
EARLY_STOPPING_PATIENCE = 10  # Dừng nếu không cải thiện sau N epochs
RESUME_TRAINING = False  # Resume từ checkpoint nếu có

In [2]:
# =============================================================================
# CẤU HÌNH MÔI TRƯỜNG
# =============================================================================
print("="*70)
print("CÀI ĐẶT THƯ VIỆN CHO GOOGLE COLAB")
print("="*70)

print("📦 Đang cài đặt PyTorch với CUDA 11.8...")
!pip install -q torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118

print("📦 Đang cài đặt transformers và timm...")
!pip install -q transformers timm

print("📦 Đang cài đặt OpenCV, PIL, và các thư viện xử lý ảnh...")
!pip install -q opencv-python pillow numpy pandas matplotlib seaborn scikit-learn scikit-image

print("📦 Đang cài đặt tqdm cho progress bar...")
!pip install -q tqdm

print("✓ Hoàn thành cài đặt thư viện!\n")

CÀI ĐẶT THƯ VIỆN CHO GOOGLE COLAB
📦 Đang cài đặt PyTorch với CUDA 11.8...
📦 Đang cài đặt transformers và timm...
📦 Đang cài đặt OpenCV, PIL, và các thư viện xử lý ảnh...
📦 Đang cài đặt tqdm cho progress bar...
✓ Hoàn thành cài đặt thư viện!



In [3]:
# =============================================================================
# IMPORT THƯ VIỆN CƠ BẢN
# =============================================================================

import os
import json
import time
import warnings
from pathlib import Path
from typing import List, Dict, Tuple, Optional
from collections import Counter
from datetime import datetime

warnings.filterwarnings('ignore')

# =============================================================================
# IMPORT THƯ VIỆN XỬ LÝ ẢNH
# =============================================================================

import cv2
import numpy as np
import pandas as pd
from PIL import Image

# =============================================================================
# IMPORT THƯ VIỆN VISUALIZATION
# =============================================================================

import matplotlib.pyplot as plt
import seaborn as sns

# Cấu hình matplotlib
%matplotlib inline
plt.rcParams['figure.figsize'] = (12, 8)
plt.rcParams['font.size'] = 10
plt.rcParams['axes.grid'] = True
plt.rcParams['grid.alpha'] = 0.3


# =============================================================================
# IMPORT THƯ VIỆN DEEP LEARNING
# =============================================================================

# =============================================================================
# IMPORT THƯ VIỆN DEEP LEARNING
# =============================================================================

# Xử lý lỗi import torch (AttributeError: partially initialized module)
try:
    import torch
    import torch.nn as nn
    import torch.optim as optim
    from torch.utils.data import Dataset, DataLoader, random_split
    from torchvision import transforms
    import torchvision.transforms.functional as TF
except AttributeError as e:
    if "partially initialized module 'torch'" in str(e) or "'torch' has no attribute" in str(e):
        print("="*70)
        print("⚠️  LỖI: CIRCULAR IMPORT HOẶC CORRUPT TORCH MODULE")
        print("="*70)
        print("Đang thử fix...")

        # Xóa cache
        import sys
        modules_to_remove = [k for k in sys.modules.keys() if 'torch' in k]
        for module in modules_to_remove:
            if 'torch' in module:
                del sys.modules[module]

        # Reinstall PyTorch
        print("📦 Đang reinstall PyTorch...")
        get_ipython().system('pip uninstall -y torch torchvision torchaudio')
        get_ipython().system('pip install -q torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118 --no-cache-dir')

        # Thử import lại
        import torch
        import torch.nn as nn
        import torch.optim as optim
        from torch.utils.data import Dataset, DataLoader, random_split
        from torchvision import transforms
        import torchvision.transforms.functional as TF
        print("✓ Đã fix và import torch thành công!")
    else:
        raise

# Kiểm tra GPU sau khi import torch
print("="*70)
print("KIỂM TRA GPU")
print("="*70)
print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"CUDA version: {torch.version.cuda}")
    gpu_memory = torch.cuda.get_device_properties(0).total_memory / 1024**3
    print(f"GPU Memory: {gpu_memory:.2f} GB")
else:
    print("⚠️  CUDA không khả dụng! Hãy đảm bảo Runtime -> Change runtime type -> GPU được chọn")
print("="*70 + "\n")

# =============================================================================
# IMPORT VISION TRANSFORMER
# =============================================================================

# Import transformers
try:
    from transformers import ViTModel, ViTConfig
    TRANSFORMERS_AVAILABLE = True
except ImportError:
    print("⚠️  transformers chưa được cài đặt, đang cài...")
    !pip install -q transformers
    from transformers import ViTModel, ViTConfig
    TRANSFORMERS_AVAILABLE = True

# Import timm
try:
    import timm
    TIMM_AVAILABLE = True
except ImportError:
    print("⚠️  timm chưa được cài đặt, đang cài...")
    !pip install -q timm
    import timm
    TIMM_AVAILABLE = True

# =============================================================================
# IMPORT METRICS VÀ UTILITIES
# =============================================================================

from sklearn.metrics import (
    classification_report,
    confusion_matrix,
    f1_score,
    precision_score,
    recall_score
)
from scipy import ndimage

# Progress bar
try:
    from tqdm import tqdm
    from tqdm.notebook import tqdm as tqdm_notebook
    USE_TQDM = True
except ImportError:
    USE_TQDM = False
    print("⚠️  tqdm chưa được cài đặt")

# =============================================================================
# HIỂN THỊ THÔNG TIN MÔI TRƯỜNG
# =============================================================================

print("="*70)
print("THÔNG TIN MÔI TRƯỜNG")
print("="*70)
print(f"✓ Device: {'CUDA' if torch.cuda.is_available() else 'CPU'}")
print(f"✓ PyTorch: {torch.__version__}")
print(f"✓ Transformers: {TRANSFORMERS_AVAILABLE}")
print(f"✓ TIMM: {TIMM_AVAILABLE}")
print(f"✓ TQDM: {USE_TQDM}")
print("="*70)
print("✓ Đã import tất cả thư viện thành công!\n")

KIỂM TRA GPU
PyTorch version: 2.8.0+cu126
CUDA available: True
GPU: Tesla T4
CUDA version: 12.6
GPU Memory: 14.74 GB

THÔNG TIN MÔI TRƯỜNG
✓ Device: CUDA
✓ PyTorch: 2.8.0+cu126
✓ Transformers: True
✓ TIMM: True
✓ TQDM: True
✓ Đã import tất cả thư viện thành công!



1.1. Cấu hình đường dẫn dữ liệu

In [ ]:
# =============================================================================
# CẤU HÌNH ĐƯỜNG DẪN DỮ LIỆU
# =============================================================================

print("="*70)
print("CẤU HÌNH GOOGLE DRIVE")
print("="*70)

# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive', force_remount=False)

print("\n📁 Chọn một trong các phương án sau:")
print("   1. Dữ liệu trên Google Drive: /content/drive/MyDrive/ViT/datasets-sam-vit")
print("   2. Upload trực tiếp lên Colab: /content/ViT/datasets-sam-vit")

# Tự động phát hiện đường dẫn
drive_path = Path("/content/drive/MyDrive/ViT/datasets-sam-vit")

if drive_path.exists() and (drive_path / "images").exists():
    BASE_DATA_DIR = drive_path
    print(f"✓ Tìm thấy dữ liệu trên Google Drive: {BASE_DATA_DIR}")
else:
    print("⚠️  Chưa tìm thấy dữ liệu!")

# Các thư mục con
IMAGES_DIR = BASE_DATA_DIR / "images"
MASKS_DIR = BASE_DATA_DIR / "masks"

# Mapping class IDs (từ mask values sang tên class)
CLASS_MAPPING = {
    0: "background",
    1: "brown_planthopper",      # Rầy nâu (BPH) - Category ID 1
    2: "whitebacked_planthopper", # Rầy lưng trắng (WBPH) - Category ID 2
    3: "rice_leaf_miner"         # Sâu ăn lá lúa (RLM) - Category ID 3
}

NUM_CLASSES = len([k for k in CLASS_MAPPING.keys() if k > 0])  # Bỏ background: 3 classes

print(f"\n{'='*70}")
print("KIỂM TRA ĐƯỜNG DẪN DỮ LIỆU")
print(f"{'='*70}")
print(f"BASE_DATA_DIR: {BASE_DATA_DIR}")
print(f"IMAGES_DIR: {IMAGES_DIR} ({'✓' if IMAGES_DIR.exists() else '✗'})")
print(f"MASKS_DIR: {MASKS_DIR} ({'✓' if MASKS_DIR.exists() else '✗'})")
print(f"\nSố classes: {NUM_CLASSES}")
print(f"Class mapping:")
for k, v in CLASS_MAPPING.items():
    if k > 0:
        print(f"  {k}: {v}")

if not IMAGES_DIR.exists() or not MASKS_DIR.exists():
    print(f"\n⚠️  CẢNH BÁO: Thư mục dữ liệu không tồn tại!")
    print("   Vui lòng kiểm tra lại đường dẫn hoặc upload dữ liệu.")


CẤU HÌNH GOOGLE DRIVE
Mounted at /content/drive

📁 Chọn một trong các phương án sau:
   1. Dữ liệu trên Google Drive: /content/drive/MyDrive/ViT/datasets-sam-vit
   2. Upload trực tiếp lên Colab: /content/ViT/datasets-sam-vit
✓ Tìm thấy dữ liệu trên Google Drive: /content/drive/MyDrive/ViT/datasets-sam-vit

KIỂM TRA ĐƯỜNG DẪN DỮ LIỆU
BASE_DATA_DIR: /content/drive/MyDrive/ViT/datasets-sam-vit
IMAGES_DIR: /content/drive/MyDrive/ViT/datasets-sam-vit/images (✓)
MASKS_DIR: /content/drive/MyDrive/ViT/datasets-sam-vit/masks (✓)
LABELS_DIR: /content/drive/MyDrive/ViT/datasets-sam-vit/labels (✗)

Số classes: 3
Class mapping:
  1: brown_planthopper
  2: whitebacked_planthopper
  3: rice_leaf_miner


## 1.2. Đọc và phân tích dữ liệu (images và masks)

In [ ]:
def analyze_dataset(images_dir: Path, masks_dir: Path, splits: List[str] = ['train', 'val', 'test']):
    """
    Phân tích dataset: đếm số lượng ảnh, masks, classes theo từng split
    """
    results = {}

    for split in splits:
        split_images_dir = images_dir / split
        split_masks_dir = masks_dir / split

        if not split_images_dir.exists() or not split_masks_dir.exists():
            print(f"⚠️  Thư mục {split} không tồn tại, bỏ qua...")
            continue

        # Tìm tất cả file ảnh
        image_files = list(split_images_dir.glob("*.jpg")) + list(split_images_dir.glob("*.JPG")) + \
                     list(split_images_dir.glob("*.png")) + list(split_images_dir.glob("*.PNG"))

        # Đếm masks và classes
        mask_files = list(split_masks_dir.glob("*.png"))

        # Đếm classes trong masks
        class_counts = Counter()
        total_mask_pixels = 0
        image_sizes = []
        valid_pairs = 0

        for img_file in image_files:
            mask_file = split_masks_dir / f"{img_file.stem}.png"
            if mask_file.exists():
                valid_pairs += 1
                mask = cv2.imread(str(mask_file), cv2.IMREAD_GRAYSCALE)
                if mask is not None:
                    unique_classes = np.unique(mask)
                    for cls in unique_classes:
                        if cls > 0:  # Bỏ background
                            class_counts[cls] += np.sum(mask == cls)
                            total_mask_pixels += np.sum(mask == cls)

                    # Đọc kích thước ảnh
                    img = cv2.imread(str(img_file))
                    if img is not None:
                        h, w = img.shape[:2]
                        image_sizes.append((w, h))

        results[split] = {
            'total_images': len(image_files),
            'total_masks': len(mask_files),
            'valid_pairs': valid_pairs,
            'class_counts': dict(class_counts),
            'total_mask_pixels': total_mask_pixels,
            'image_sizes': image_sizes
        }

        print(f"\n{'='*70}")
        print(f"Split: {split.upper()}")
        print(f"{'='*70}")
        print(f"Tổng số ảnh: {results[split]['total_images']}")
        print(f"Tổng số masks: {results[split]['total_masks']}")
        print(f"Số cặp ảnh-mask hợp lệ: {results[split]['valid_pairs']}")
        print(f"Phân bố classes (pixels):")
        for cls_id, count in sorted(results[split]['class_counts'].items()):
            cls_name = CLASS_MAPPING.get(cls_id, f"Unknown_{cls_id}")
            percentage = (count / total_mask_pixels * 100) if total_mask_pixels > 0 else 0
            print(f"  - {cls_name} (ID={cls_id}): {count:,} pixels ({percentage:.2f}%)")

        if image_sizes:
            sizes_array = np.array(image_sizes)
            print(f"\nKích thước ảnh (W x H):")
            print(f"  - Trung bình: {sizes_array[:, 0].mean():.0f} x {sizes_array[:, 1].mean():.0f}")
            print(f"  - Min: {sizes_array[:, 0].min()} x {sizes_array[:, 1].min()}")
            print(f"  - Max: {sizes_array[:, 0].max()} x {sizes_array[:, 1].max()}")

    return results

# Phân tích dataset
dataset_stats = analyze_dataset(IMAGES_DIR, MASKS_DIR)


## 1.3. Visualize một số mẫu dữ liệu


In [ ]:
def visualize_samples(images_dir: Path, masks_dir: Path, split: str = 'train', num_samples: int = 6):
    """
    Hiển thị một số mẫu ảnh và mask tương ứng
    """
    split_images_dir = images_dir / split
    split_masks_dir = masks_dir / split

    if not split_images_dir.exists() or not split_masks_dir.exists():
        print(f"⚠️  Thư mục {split} không tồn tại")
        return

    # Tìm các file ảnh có mask tương ứng
    image_files = list(split_images_dir.glob("*.jpg")) + list(split_images_dir.glob("*.png"))
    valid_pairs = []

    for img_file in image_files[:50]:  # Chỉ kiểm tra 50 file đầu
        mask_file = split_masks_dir / f"{img_file.stem}.png"
        if mask_file.exists():
            valid_pairs.append((img_file, mask_file))
        if len(valid_pairs) >= num_samples:
            break

    if len(valid_pairs) == 0:
        print(f"⚠️  Không tìm thấy cặp ảnh-mask hợp lệ trong split {split}")
        return

    # Tạo figure
    fig, axes = plt.subplots(num_samples, 3, figsize=(15, 5*num_samples))
    if num_samples == 1:
        axes = axes.reshape(1, -1)

    # Color map cho mask visualization (RGB)
    mask_color_map = {
        0: (0, 0, 0),       # Background - đen
        1: (255, 0, 0),     # Brown Planthopper - đỏ
        2: (0, 255, 0),     # White-Backed Planthopper - xanh lá
        3: (0, 0, 255),     # Rice Leaf Miner - xanh dương
    }

    for idx, (img_file, mask_file) in enumerate(valid_pairs[:num_samples]):
        # Đọc ảnh
        img = cv2.imread(str(img_file))
        img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

        # Đọc mask
        mask = cv2.imread(str(mask_file), cv2.IMREAD_GRAYSCALE)

        # Tạo mask visualization màu
        mask_vis = np.zeros((mask.shape[0], mask.shape[1], 3), dtype=np.uint8)
        for cls_id, color in mask_color_map.items():
            mask_vis[mask == cls_id] = color

        # Overlay mask lên ảnh
        overlay = img_rgb.copy()
        mask_colored = mask_vis.copy()
        mask_colored[mask == 0] = 0  # Chỉ overlay các pixel không phải background
        overlay = cv2.addWeighted(overlay, 0.7, mask_colored, 0.3, 0)

        # Hiển thị
        axes[idx, 0].imshow(img_rgb)
        axes[idx, 0].set_title(f"Ảnh gốc: {img_file.name}")
        axes[idx, 0].axis('off')

        axes[idx, 1].imshow(mask_vis)
        axes[idx, 1].set_title(f"Mask (ID values)")
        axes[idx, 1].axis('off')

        # Hiển thị các classes có trong mask
        unique_classes = [c for c in np.unique(mask) if c > 0]
        classes_text = ", ".join([f"{CLASS_MAPPING[c]}({c})" for c in unique_classes])

        axes[idx, 2].imshow(overlay)
        axes[idx, 2].set_title(f"Overlay\nClasses: {classes_text}")
        axes[idx, 2].axis('off')

    plt.tight_layout()
    plt.show()

# Visualize samples
visualize_samples(IMAGES_DIR, MASKS_DIR, split='train', num_samples=6)


## 1.4. Dataset class để load dữ liệu cho ViT

Dataset này sẽ:
- Đọc ảnh gốc và mask tương ứng
- Trích xuất các vùng ROI từ mask (SAM đã tạo mask chính xác)
- Crop và resize các ROI về kích thước chuẩn cho ViT
- Áp dụng augmentation nếu cần

In [ ]:
class PlanthopperDataset(Dataset):
    """
    Dataset class cho phân loại rầy nâu, rầy lưng trắng và sâu ăn lá lúa
    Đọc ảnh đã được mask (từ SAM2) và trích xuất ROI cho ViT
    """
    def __init__(
        self,
        images_dir: Path,
        masks_dir: Path,
        split: str,
        image_size: int = 224,
        augment: bool = False,
        extract_roi: bool = True,
        min_roi_size: int = 32,
    ):
        self.images_dir = images_dir / split
        self.masks_dir = masks_dir / split
        self.split = split
        self.image_size = image_size
        self.augment = augment and (split == 'train')
        self.extract_roi = extract_roi
        self.min_roi_size = min_roi_size

        # Tìm tất cả các cặp ảnh-mask hợp lệ
        image_files = list(self.images_dir.glob("*.jpg")) + list(self.images_dir.glob("*.png")) + \
                     list(self.images_dir.glob("*.JPG")) + list(self.images_dir.glob("*.PNG"))

        self.samples = []
        for img_file in image_files:
            mask_file = self.masks_dir / f"{img_file.stem}.png"
            if mask_file.exists():
                # Kiểm tra mask có dữ liệu không
                mask = cv2.imread(str(mask_file), cv2.IMREAD_GRAYSCALE)
                if mask is not None and np.sum(mask > 0) > 0:
                    if extract_roi:
                        # Trích xuất từng ROI riêng biệt
                        rois = self._extract_rois(img_file, mask_file, mask)
                        self.samples.extend(rois)
                    else:
                        # Sử dụng toàn bộ ảnh
                        self.samples.append({
                            'image_file': img_file,
                            'mask_file': mask_file,
                            'roi_bbox': None,  # Toàn bộ ảnh
                            'label': None  # Sẽ xác định từ mask
                        })

        print(f"✓ Split {split}: {len(self.samples)} samples")

        # Augmentation transforms
        if self.augment:
            self.aug_transform = transforms.Compose([
                transforms.RandomHorizontalFlip(p=0.5),
                transforms.RandomRotation(degrees=15),
                transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2),
            ])
        else:
            self.aug_transform = None

        # Normalization (ImageNet mean/std)
        self.normalize = transforms.Normalize(
            mean=[0.485, 0.456, 0.406],
            std=[0.229, 0.224, 0.225]
        )

    def _extract_rois(self, image_file: Path, mask_file: Path, mask: np.ndarray) -> List[Dict]:
        """
        Trích xuất các ROI (Regions of Interest) từ mask
        Mỗi ROI tương ứng với một instance của một class
        """
        rois = []
        unique_classes = [c for c in np.unique(mask) if c > 0]  # Bỏ background

        for class_id in unique_classes:
            # Tạo binary mask cho class này
            class_mask = (mask == class_id).astype(np.uint8)

            # Tìm contours để tách các instance riêng biệt
            contours, _ = cv2.findContours(class_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

            for contour in contours:
                area = cv2.contourArea(contour)
                if area < self.min_roi_size * self.min_roi_size:  # Bỏ qua ROI quá nhỏ
                    continue

                # Lấy bounding box
                x, y, w, h = cv2.boundingRect(contour)

                # Mở rộng bbox một chút để có context
                padding = 10
                x = max(0, x - padding)
                y = max(0, y - padding)
                w = min(mask.shape[1] - x, w + 2 * padding)
                h = min(mask.shape[0] - y, h + 2 * padding)

                rois.append({
                    'image_file': image_file,
                    'mask_file': mask_file,
                    'roi_bbox': (x, y, w, h),
                    'label': class_id - 1  # Chuyển từ [1,2,3] sang [0,1,2] cho ViT
                })

        return rois

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        sample = self.samples[idx]

        # Đọc ảnh
        img = cv2.imread(str(sample['image_file']))
        img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

        if sample['roi_bbox'] is not None:
            # Crop ROI
            x, y, w, h = sample['roi_bbox']
            roi = img_rgb[y:y+h, x:x+w]
        else:
            # Sử dụng toàn bộ ảnh
            roi = img_rgb

        # Resize về image_size
        roi_pil = Image.fromarray(roi)
        roi_resized = roi_pil.resize((self.image_size, self.image_size), Image.BILINEAR)

        # Convert to tensor
        img_tensor = TF.to_tensor(roi_resized)

        # Augmentation
        if self.aug_transform is not None:
            img_tensor = self.aug_transform(img_tensor)

        # Normalize
        img_tensor = self.normalize(img_tensor)

        # Label
        label = sample['label']
        if label is None:
            # Xác định label từ mask nếu chưa có
            mask = cv2.imread(str(sample['mask_file']), cv2.IMREAD_GRAYSCALE)
            unique_classes = [c for c in np.unique(mask) if c > 0]
            label = unique_classes[0] - 1 if unique_classes else 0

        return img_tensor, torch.tensor(label, dtype=torch.long)

# Test dataset
print("Tạo datasets...")
train_dataset = PlanthopperDataset(IMAGES_DIR, MASKS_DIR, 'train', augment=True)
val_dataset = PlanthopperDataset(IMAGES_DIR, MASKS_DIR, 'val', augment=False)
test_dataset = PlanthopperDataset(IMAGES_DIR, MASKS_DIR, 'test', augment=False)

print(f"\nTrain: {len(train_dataset)} samples")
print(f"Val: {len(val_dataset)} samples")
print(f"Test: {len(test_dataset)} samples")


In [ ]:
# =============================================================================
# LOAD SAM2 MODEL (Optional - chỉ cần nếu muốn tạo masks mới)
# =============================================================================

LOAD_SAM = False  # Đặt True nếu cần sử dụng SAM

if LOAD_SAM:
    from sam2.build_sam import build_sam2
    from sam2.sam2_image_predictor import SAM2ImagePredictor

    # Đường dẫn checkpoint và config
    SAM_CHECKPOINT = "./checkpoints/sam2.1_hiera_tiny.pt"
    SAM_MODEL_CFG = "configs/sam2.1/sam2.1_hiera_t.yaml"

    if os.path.exists(SAM_CHECKPOINT):
        print("Loading SAM2 model...")
        device = "cuda" if torch.cuda.is_available() else "cpu"
        sam_model = build_sam2(SAM_MODEL_CFG, SAM_CHECKPOINT, device=device)
        sam_predictor = SAM2ImagePredictor(sam_model)
        print(f"✓ SAM2 loaded on {device}")
    else:
        print(f"⚠️  SAM checkpoint not found at {SAM_CHECKPOINT}")
        print("   Skip loading SAM (masks đã được tạo sẵn)")
        sam_predictor = None
else:
    print("Skip loading SAM (masks đã được tạo sẵn từ bước preprocessing)")
    sam_predictor = None

## 2.2. Xây dựng Vision Transformer (ViT) Model

Sử dụng ViT để phân loại các ROI đã được trích xuất từ SAM masks.

In [ ]:
class ViTClassifier(nn.Module):
    """
    Vision Transformer classifier cho phân loại rầy
    Sử dụng pretrained ViT và thêm classification head
    """
    def __init__(
        self,
        num_classes: int = 3,
        model_name: str = 'google/vit-base-patch16-224',
        pretrained: bool = True,
        dropout: float = 0.1,
    ):
        super().__init__()

        # Load pretrained ViT
        if pretrained:
            self.vit = ViTModel.from_pretrained(model_name)
        else:
            config = ViTConfig.from_pretrained(model_name)
            self.vit = ViTModel(config)

        # Feature dimension
        hidden_size = self.vit.config.hidden_size

        # Classification head
        self.classifier = nn.Sequential(
            nn.Dropout(dropout),
            nn.Linear(hidden_size, hidden_size // 2),
            nn.GELU(),
            nn.Dropout(dropout),
            nn.Linear(hidden_size // 2, num_classes)
        )

        # Freeze một phần encoder nếu muốn (optional)
        # for param in list(self.vit.encoder.layer[:6].parameters()):
        #     param.requires_grad = False

    def forward(self, x):
        # ViT forward
        outputs = self.vit(pixel_values=x)

        # Lấy [CLS] token
        cls_token = outputs.last_hidden_state[:, 0]

        # Classification
        logits = self.classifier(cls_token)

        return logits

# Hoặc sử dụng timm (alternative)
class ViTClassifierTimm(nn.Module):
    """
    ViT classifier sử dụng timm library
    """
    def __init__(
        self,
        num_classes: int = 3,
        model_name: str = 'vit_base_patch16_224',
        pretrained: bool = True,
        dropout: float = 0.1,
    ):
        super().__init__()

        # Load pretrained ViT từ timm
        self.backbone = timm.create_model(
            model_name,
            pretrained=pretrained,
            num_classes=0,  # Chỉ lấy features
        )

        feature_dim = self.backbone.num_features

        # Classification head
        self.classifier = nn.Sequential(
            nn.Dropout(dropout),
            nn.Linear(feature_dim, feature_dim // 2),
            nn.GELU(),
            nn.Dropout(dropout),
            nn.Linear(feature_dim // 2, num_classes)
        )

    def forward(self, x):
        features = self.backbone(x)
        logits = self.classifier(features)
        return logits

# Tạo model
print("Tạo ViT model...")
model = ViTClassifierTimm(
    num_classes=NUM_CLASSES,
    model_name='vit_base_patch16_224',  # Có thể thay bằng 'vit_small_patch16_224' cho model nhỏ hơn
    pretrained=True,
    dropout=0.1
)

# Test forward pass
dummy_input = torch.randn(1, 3, 224, 224)
with torch.no_grad():
    output = model(dummy_input)
print(f"✓ Model created!")
print(f"  Input shape: {dummy_input.shape}")
print(f"  Output shape: {output.shape}")
print(f"  Number of parameters: {sum(p.numel() for p in model.parameters()):,}")
print(f"  Trainable parameters: {sum(p.numel() for p in model.parameters() if p.requires_grad):,}")

In [ ]:
# =============================================================================
# TẠO DATALOADERS
# =============================================================================

NUM_WORKERS = 0

train_loader = DataLoader(
    train_dataset,
    batch_size=BATCH_SIZE,
    shuffle=True,
    num_workers=NUM_WORKERS,
    pin_memory=True if torch.cuda.is_available() else False
)

val_loader = DataLoader(
    val_dataset,
    batch_size=BATCH_SIZE,
    shuffle=False,
    num_workers=NUM_WORKERS,
    pin_memory=True if torch.cuda.is_available() else False
)

test_loader = DataLoader(
    test_dataset,
    batch_size=BATCH_SIZE,
    shuffle=False,
    num_workers=NUM_WORKERS,
    pin_memory=True if torch.cuda.is_available() else False
)

print(f"✓ DataLoaders created!")
print(f"  Train batches: {len(train_loader)}")
print(f"  Val batches: {len(val_loader)}")
print(f"  Test batches: {len(test_loader)}")

# Kiểm tra một batch
sample_batch = next(iter(train_loader))
images, labels = sample_batch
print(f"\nSample batch:")
print(f"  Images shape: {images.shape}")
print(f"  Labels shape: {labels.shape}")
print(f"  Labels: {labels[:10].tolist()}...")  # Hiển thị 10 labels đầu

## 3.2. Hàm huấn luyện và đánh giá


In [ ]:
def train_epoch(model, train_loader, criterion, optimizer, device, epoch):
    """Huấn luyện một epoch"""
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0

    # Progress bar
    if USE_TQDM:
        pbar = tqdm_notebook(train_loader, desc=f'Epoch {epoch+1} [Train]', leave=False)
    else:
        pbar = train_loader

    for batch_idx, (images, labels) in enumerate(pbar):
        images, labels = images.to(device, non_blocking=True), labels.to(device, non_blocking=True)

        # Forward
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)

        # Backward
        loss.backward()

        # Gradient clipping để tránh gradient explosion
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)

        optimizer.step()

        # Statistics
        running_loss += loss.item()
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

        # Update progress bar
        if USE_TQDM:
            current_acc = 100. * correct / total
            pbar.set_postfix({
                'loss': f'{loss.item():.4f}',
                'acc': f'{current_acc:.2f}%'
            })
        elif (batch_idx + 1) % 50 == 0:
            print(f'  Batch [{batch_idx+1}/{len(train_loader)}], '
                  f'Loss: {loss.item():.4f}, '
                  f'Acc: {100.*correct/total:.2f}%')

    epoch_loss = running_loss / len(train_loader)
    epoch_acc = 100. * correct / total

    return epoch_loss, epoch_acc

def evaluate(model, data_loader, criterion, device, class_names=None, desc='Eval'):
    """Đánh giá model trên validation/test set"""
    model.eval()
    running_loss = 0.0
    all_preds = []
    all_labels = []

    # Progress bar
    if USE_TQDM:
        pbar = tqdm_notebook(data_loader, desc=desc, leave=False)
    else:
        pbar = data_loader

    with torch.no_grad():
        for images, labels in pbar:
            images, labels = images.to(device, non_blocking=True), labels.to(device, non_blocking=True)

            outputs = model(images)
            loss = criterion(outputs, labels)

            running_loss += loss.item()
            _, predicted = torch.max(outputs.data, 1)

            all_preds.extend(predicted.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

            if USE_TQDM:
                pbar.set_postfix({'loss': f'{loss.item():.4f}'})

    epoch_loss = running_loss / len(data_loader)
    all_preds = np.array(all_preds)
    all_labels = np.array(all_labels)

    # Tính các metrics
    accuracy = 100. * np.mean(all_preds == all_labels)
    precision = precision_score(all_labels, all_preds, average='weighted', zero_division=0)
    recall = recall_score(all_labels, all_preds, average='weighted', zero_division=0)
    f1 = f1_score(all_labels, all_preds, average='weighted', zero_division=0)

    # Per-class metrics
    per_class_metrics = {}
    if class_names is None:
        class_names = [f"Class_{i}" for i in range(len(np.unique(all_labels)))]

    unique_labels = np.unique(all_labels)
    for cls_id in unique_labels:
        cls_mask = all_labels == cls_id
        if np.sum(cls_mask) > 0:
            cls_acc = 100. * np.mean(all_preds[cls_mask] == all_labels[cls_mask])
            cls_precision = precision_score(all_labels == cls_id, all_preds == cls_id, zero_division=0)
            cls_recall = recall_score(all_labels == cls_id, all_preds == cls_id, zero_division=0)
            cls_f1 = f1_score(all_labels == cls_id, all_preds == cls_id, zero_division=0)

            per_class_metrics[class_names[cls_id]] = {
                'accuracy': cls_acc,
                'precision': cls_precision,
                'recall': cls_recall,
                'f1': cls_f1
            }

    return {
        'loss': epoch_loss,
        'accuracy': accuracy,
        'precision': precision,
        'recall': recall,
        'f1': f1,
        'predictions': all_preds,
        'labels': all_labels,
        'per_class': per_class_metrics
    }

print("✓ Training và evaluation functions đã được định nghĩa")


In [ ]:


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"{'='*70}")
print("CẤU HÌNH HUẤN LUYỆN")
print(f"{'='*70}")
print(f"Device: {device}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
print(f"Epochs: {NUM_EPOCHS}")
print(f"Learning rate: {LEARNING_RATE}")
print(f"Batch size: {BATCH_SIZE}")
print(f"Early stopping patience: {EARLY_STOPPING_PATIENCE}")
print(f"{'='*70}")

# Di chuyển model lên device
model = model.to(device)

# Loss function (CrossEntropy với class weights để xử lý class imbalance)
# Tính class weights từ dataset
print("\n📊 Tính toán class weights...")
class_counts = np.zeros(NUM_CLASSES)
print("   Đang đếm samples trong train dataset...")
for _, label in train_dataset:
    class_counts[label] += 1

total_samples = np.sum(class_counts)
class_weights = total_samples / (NUM_CLASSES * class_counts)
class_weights = torch.FloatTensor(class_weights).to(device)

print(f"   Class counts: {class_counts}")
print(f"   Class weights: {class_weights.cpu().numpy()}")
print(f"   Total samples: {total_samples}")

criterion = nn.CrossEntropyLoss(weight=class_weights)

# Optimizer
optimizer = optim.AdamW(model.parameters(), lr=LEARNING_RATE, weight_decay=WEIGHT_DECAY)

# Learning rate scheduler (CosineAnnealingLR với warmup)
warmup_epochs = 3
scheduler = optim.lr_scheduler.CosineAnnealingLR(
    optimizer,
    T_max=NUM_EPOCHS - warmup_epochs,
    eta_min=1e-6
)

# Lưu training history
history = {
    'train_loss': [],
    'train_acc': [],
    'val_loss': [],
    'val_acc': [],
    'val_f1': [],
    'learning_rates': []
}

# Best model
best_val_f1 = 0.0
best_epoch = 0
best_model_state = None
early_stopping_counter = 0

# Resume training nếu cần
start_epoch = 0
if RESUME_TRAINING:
    checkpoint_path = Path("/content/drive/MyDrive/ViT_checkpoints/vit_classifier_best.pth")
    if checkpoint_path.exists():
        print(f"\n📂 Resume training từ checkpoint: {checkpoint_path}")
        checkpoint = torch.load(checkpoint_path, map_location=device)
        model.load_state_dict(checkpoint['model_state_dict'])
        optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
        start_epoch = checkpoint.get('epoch', 0)
        best_val_f1 = checkpoint.get('best_val_f1', 0.0)
        history = checkpoint.get('history', history)
        print(f"   Resume từ epoch {start_epoch}, best F1: {best_val_f1:.4f}")

class_names = [CLASS_MAPPING[i+1] for i in range(NUM_CLASSES)]

print(f"\n{'='*70}")
print("BẮT ĐẦU HUẤN LUYỆN")
print(f"{'='*70}")
print(f"Start epoch: {start_epoch}")
print(f"Class names: {class_names}")
print(f"{'='*70}\n")

In [ ]:
# =============================================================================
# TRAINING LOOP
# =============================================================================

training_start_time = time.time()

for epoch in range(start_epoch, NUM_EPOCHS):
    epoch_start_time = time.time()

    print(f"\n{'='*70}")
    print(f"Epoch [{epoch+1}/{NUM_EPOCHS}]")
    print(f"{'='*70}")

    # Warmup learning rate cho 3 epochs đầu
    if epoch < warmup_epochs:
        warmup_lr = LEARNING_RATE * (epoch + 1) / warmup_epochs
        for param_group in optimizer.param_groups:
            param_group['lr'] = warmup_lr
    else:
        scheduler.step()

    current_lr = optimizer.param_groups[0]['lr']
    history['learning_rates'].append(current_lr)

    # Train
    train_loss, train_acc = train_epoch(model, train_loader, criterion, optimizer, device, epoch)

    # Validate
    val_results = evaluate(model, val_loader, criterion, device, class_names=class_names, desc=f'Epoch {epoch+1} [Val]')

    # Save history
    history['train_loss'].append(train_loss)
    history['train_acc'].append(train_acc)
    history['val_loss'].append(val_results['loss'])
    history['val_acc'].append(val_results['accuracy'])
    history['val_f1'].append(val_results['f1'])

    # Print results
    epoch_time = time.time() - epoch_start_time
    print(f"\nTrain - Loss: {train_loss:.4f}, Acc: {train_acc:.2f}%")
    print(f"Val   - Loss: {val_results['loss']:.4f}, "
          f"Acc: {val_results['accuracy']:.2f}%, "
          f"F1: {val_results['f1']:.4f}")
    print(f"      - Precision: {val_results['precision']:.4f}, "
          f"Recall: {val_results['recall']:.4f}")
    print(f"      - LR: {current_lr:.6f}, Time: {epoch_time:.1f}s")

    # Print per-class metrics
    print("\n      Per-class metrics:")
    for cls_name, metrics in val_results['per_class'].items():
        print(f"        {cls_name}: "
              f"Acc={metrics['accuracy']:.2f}%, "
              f"P={metrics['precision']:.4f}, "
              f"R={metrics['recall']:.4f}, "
              f"F1={metrics['f1']:.4f}")

    # Save best model
    improved = False
    if val_results['f1'] > best_val_f1:
        best_val_f1 = val_results['f1']
        best_epoch = epoch + 1
        best_model_state = model.state_dict().copy()
        early_stopping_counter = 0
        improved = True
        print(f"\n      ✅ New best F1: {best_val_f1:.4f} (Epoch {best_epoch})")

        # Lưu checkpoint trên Colab
        checkpoint_dir = Path("/content/drive/MyDrive/ViT_checkpoints")
        checkpoint_dir.mkdir(parents=True, exist_ok=True)

        checkpoint = {
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'scheduler_state_dict': scheduler.state_dict(),
            'epoch': epoch + 1,
            'best_val_f1': best_val_f1,
            'best_epoch': best_epoch,
            'test_results': val_results,
            'class_names': class_names,
            'history': history,
            'config': {
                'num_classes': NUM_CLASSES,
                'learning_rate': LEARNING_RATE,
                'batch_size': BATCH_SIZE,
                'class_mapping': CLASS_MAPPING
            }
        }

        checkpoint_path = checkpoint_dir / 'vit_classifier_best.pth'
        torch.save(checkpoint, checkpoint_path)
        print(f"      💾 Saved checkpoint: {checkpoint_path}")
    else:
        early_stopping_counter += 1
        print(f"\n      ⏳ No improvement ({early_stopping_counter}/{EARLY_STOPPING_PATIENCE})")

    # Early stopping
    if early_stopping_counter >= EARLY_STOPPING_PATIENCE:
        print(f"\n{'='*70}")
        print(f"⏹️  EARLY STOPPING")
        print(f"{'='*70}")
        print(f"Không cải thiện sau {EARLY_STOPPING_PATIENCE} epochs")
        print(f"Best F1: {best_val_f1:.4f} tại epoch {best_epoch}")
        break

# Load best model
if best_model_state is not None:
    model.load_state_dict(best_model_state)
    print(f"\n✓ Loaded best model (F1: {best_val_f1:.4f} tại epoch {best_epoch})")

total_training_time = time.time() - training_start_time
print(f"\n{'='*70}")
print("HUẤN LUYỆN HOÀN TẤT")
print(f"{'='*70}")
print(f"Total training time: {total_training_time/60:.1f} minutes")
print(f"Best validation F1: {best_val_f1:.4f}")
print(f"Best epoch: {best_epoch}")
print(f"{'='*70}")


## 3.4. Visualize training history


In [ ]:
# Vẽ đồ thị training history
fig, axes = plt.subplots(1, 3, figsize=(18, 5))

# Loss
axes[0].plot(history['train_loss'], label='Train Loss', marker='o')
axes[0].plot(history['val_loss'], label='Val Loss', marker='s')
axes[0].set_xlabel('Epoch')
axes[0].set_ylabel('Loss')
axes[0].set_title('Training and Validation Loss')
axes[0].legend()
axes[0].grid(True)

# Accuracy
axes[1].plot(history['train_acc'], label='Train Acc', marker='o')
axes[1].plot(history['val_acc'], label='Val Acc', marker='s')
axes[1].set_xlabel('Epoch')
axes[1].set_ylabel('Accuracy (%)')
axes[1].set_title('Training and Validation Accuracy')
axes[1].legend()
axes[1].grid(True)

# F1 Score
axes[2].plot(history['val_f1'], label='Val F1', marker='s', color='green')
axes[2].set_xlabel('Epoch')
axes[2].set_ylabel('F1 Score')
axes[2].set_title('Validation F1 Score')
axes[2].legend()
axes[2].grid(True)

plt.tight_layout()
plt.show()


## 3.5. Đánh giá trên test set


In [ ]:
# Đánh giá trên test set
print("="*70)
print("ĐÁNH GIÁ TRÊN TEST SET")
print("="*70)

test_results = evaluate(model, test_loader, criterion, device, class_names=class_names, desc='Test')

print(f"\nTest Results:")
print(f"  Loss: {test_results['loss']:.4f}")
print(f"  Accuracy: {test_results['accuracy']:.2f}%")
print(f"  Precision: {test_results['precision']:.4f}")
print(f"  Recall: {test_results['recall']:.4f}")
print(f"  F1 Score: {test_results['f1']:.4f}")

print(f"\nPer-class metrics:")
for cls_name, metrics in test_results['per_class'].items():
    print(f"  {cls_name}:")
    print(f"    Accuracy: {metrics['accuracy']:.2f}%")
    print(f"    Precision: {metrics['precision']:.4f}")
    print(f"    Recall: {metrics['recall']:.4f}")
    print(f"    F1: {metrics['f1']:.4f}")

# Classification report
print(f"\nClassification Report:")
print(classification_report(
    test_results['labels'],
    test_results['predictions'],
    target_names=class_names
))

# Confusion matrix
cm = confusion_matrix(test_results['labels'], test_results['predictions'])
plt.figure(figsize=(10, 8))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
            xticklabels=class_names, yticklabels=class_names)
plt.title('Confusion Matrix - Test Set (SAM-ViT)', fontsize=14, fontweight='bold')
plt.ylabel('True Label', fontsize=12)
plt.xlabel('Predicted Label', fontsize=12)
plt.tight_layout()
plt.show()


# 4. PHÂN TÍCH KẾT QUẢ, XÁC ĐỊNH MẬT ĐỘ VÀ SO SÁNH

## 4.1. Tính IoU và Dice Coefficient cho Segmentation Masks

Tính toán IoU (Intersection over Union) và Dice coefficient để đánh giá chất lượng segmentation masks từ SAM.

In [ ]:
# Visualize một số mẫu dự đoán
def visualize_predictions(model, test_loader, device, num_samples=12):
    """Hiển thị một số mẫu dự đoán trên test set"""
    model.eval()

    # Lấy một batch từ test_loader
    dataiter = iter(test_loader)
    images, labels = next(dataiter)
    images = images.to(device)

    with torch.no_grad():
        outputs = model(images)
        _, predicted = torch.max(outputs, 1)
        probabilities = torch.nn.functional.softmax(outputs, dim=1)

    images = images.cpu()
    labels = labels.cpu().numpy()
    predicted = predicted.cpu().numpy()
    probabilities = probabilities.cpu().numpy()

    # Tạo figure
    fig, axes = plt.subplots(3, 4, figsize=(16, 12))
    axes = axes.flatten()

    class_names = [CLASS_MAPPING[i+1] for i in range(NUM_CLASSES)]

    for idx in range(min(num_samples, len(images))):
        img = images[idx]
        true_label = labels[idx]
        pred_label = predicted[idx]
        prob = probabilities[idx]

        # Denormalize image
        mean = torch.tensor([0.485, 0.456, 0.406]).view(3, 1, 1)
        std = torch.tensor([0.229, 0.224, 0.225]).view(3, 1, 1)
        img_denorm = img * std + mean
        img_denorm = torch.clamp(img_denorm, 0, 1)
        img_np = img_denorm.permute(1, 2, 0).numpy()

        # Hiển thị
        axes[idx].imshow(img_np)

        # Title với thông tin prediction
        true_name = class_names[true_label]
        pred_name = class_names[pred_label]
        correct = "✓" if true_label == pred_label else "✗"

        title = f"{correct} True: {true_name}\nPred: {pred_name} ({prob[pred_label]:.2f})"
        color = 'green' if true_label == pred_label else 'red'

        axes[idx].set_title(title, color=color, fontsize=9, fontweight='bold')
        axes[idx].axis('off')

    plt.suptitle('Visualization: Predictions trên Test Set (SAM-ViT)',
                 fontsize=14, fontweight='bold', y=0.995)
    plt.tight_layout()
    plt.show()

# Visualize predictions
print("\nVisualizing predictions...")
visualize_predictions(model, test_loader, device, num_samples=12)

In [ ]:
# =============================================================================
# TÍNH IOU VÀ DICE COEFFICIENT CHO SEGMENTATION MASKS
# =============================================================================

def calculate_iou(mask1: np.ndarray, mask2: np.ndarray) -> float:
    """
    Tính IoU (Intersection over Union) giữa hai masks
    """
    intersection = np.logical_and(mask1, mask2).sum()
    union = np.logical_or(mask1, mask2).sum()
    if union == 0:
        return 1.0 if intersection == 0 else 0.0
    return intersection / union

def calculate_dice_coefficient(mask1: np.ndarray, mask2: np.ndarray) -> float:
    """
    Tính Dice coefficient giữa hai masks
    Dice = 2 * |A ∩ B| / (|A| + |B|)
    """
    intersection = np.logical_and(mask1, mask2).sum()
    mask1_area = mask1.sum()
    mask2_area = mask2.sum()
    if mask1_area + mask2_area == 0:
        return 1.0 if intersection == 0 else 0.0
    return 2.0 * intersection / (mask1_area + mask2_area)

def evaluate_segmentation_masks(masks_dir: Path, split: str = 'test', class_ids: List[int] = None):
    """
    Đánh giá chất lượng segmentation masks từ SAM

    Note: Trong trường hợp này, chúng ta chỉ có masks từ SAM (không có ground truth masks để so sánh),
    nên sẽ đánh giá chất lượng masks dựa trên:
    - Số lượng pixels được segment cho mỗi class
    - Độ tách biệt giữa các instances
    - Phân bố mật độ
    """
    split_masks_dir = masks_dir / split

    if not split_masks_dir.exists():
        print(f"⚠️  Thư mục {split_masks_dir} không tồn tại")
        return None

    mask_files = list(split_masks_dir.glob("*.png"))
    # Bỏ qua các file _vis.png
    mask_files = [f for f in mask_files if not f.name.endswith('_vis.png')]

    if class_ids is None:
        class_ids = [1, 2, 3]  # Brown, White-backed, Rice leaf miner

    results = {
        'total_masks': len(mask_files),
        'class_stats': {},
        'average_mask_quality': {}
    }

    all_iou_per_class = {cls_id: [] for cls_id in class_ids}
    all_dice_per_class = {cls_id: [] for cls_id in class_ids}

    print(f"Đang đánh giá {len(mask_files)} masks từ split {split}...")

    for mask_file in tqdm_notebook(mask_files, desc='Evaluating masks', leave=False) if USE_TQDM else mask_files:
        mask = cv2.imread(str(mask_file), cv2.IMREAD_GRAYSCALE)
        if mask is None:
            continue

        # Tính toán cho từng class
        for class_id in class_ids:
            class_mask = (mask == class_id).astype(np.uint8)

            if class_mask.sum() == 0:
                continue

            # Tách các instances riêng biệt
            contours, _ = cv2.findContours(class_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

            if len(contours) > 1:
                # Tính IoU và Dice giữa các instances (để đánh giá độ tách biệt)
                # Tạo binary mask cho mỗi instance và so sánh với toàn bộ class mask
                for i, contour1 in enumerate(contours[:5]):  # Giới hạn 5 instances đầu
                    instance_mask1 = np.zeros_like(class_mask)
                    cv2.fillPoly(instance_mask1, [contour1], 1)

                    # So sánh instance với toàn bộ class mask
                    iou = calculate_iou(instance_mask1, class_mask)
                    dice = calculate_dice_coefficient(instance_mask1, class_mask)

                    all_iou_per_class[class_id].append(iou)
                    all_dice_per_class[class_id].append(dice)

    # Tính thống kê
    for class_id in class_ids:
        cls_name = CLASS_MAPPING.get(class_id, f"Class_{class_id}")
        if len(all_iou_per_class[class_id]) > 0:
            results['class_stats'][cls_name] = {
                'mean_iou': np.mean(all_iou_per_class[class_id]),
                'mean_dice': np.mean(all_dice_per_class[class_id]),
                'num_instances': len(all_iou_per_class[class_id])
            }

    return results

# Đánh giá masks
print("="*70)
print("ĐÁNH GIÁ CHẤT LƯỢNG SEGMENTATION MASKS")
print("="*70)

segmentation_results = {}
for split in ['train', 'val', 'test']:
    print(f"\nĐang đánh giá split: {split}")
    results = evaluate_segmentation_masks(MASKS_DIR, split=split)
    if results:
        segmentation_results[split] = results

# Hiển thị kết quả
print(f"\n{'='*70}")
print("KẾT QUẢ ĐÁNH GIÁ SEGMENTATION MASKS")
print(f"{'='*70}")

for split, results in segmentation_results.items():
    print(f"\nSplit: {split.upper()}")
    print(f"  Tổng số masks: {results['total_masks']}")
    print(f"  Thống kê theo class:")
    for cls_name, stats in results['class_stats'].items():
        print(f"    {cls_name}:")
        print(f"      Mean IoU: {stats['mean_iou']:.4f}")
        print(f"      Mean Dice: {stats['mean_dice']:.4f}")
        print(f"      Số instances: {stats['num_instances']}")


## 4.2. Xác định mật độ rầy nâu từ Masks

Tính toán mật độ rầy nâu (số lượng cá thể/đơn vị diện tích) từ segmentation masks để hỗ trợ đánh giá mức độ xâm nhiễm.


In [ ]:
# =============================================================================
# XÁC ĐỊNH MẬT ĐỘ RẦY NÂU TỪ MASKS
# =============================================================================

def calculate_planthopper_density(masks_dir: Path, images_dir: Path, split: str = 'test'):
    """
    Tính mật độ rầy nâu từ segmentation masks

    Mật độ được tính bằng:
    - Số lượng instances (cá thể) / ảnh
    - Số lượng pixels / diện tích ảnh (pixels/cm² nếu có thông tin scale)
    """
    split_masks_dir = masks_dir / split
    split_images_dir = images_dir / split

    if not split_masks_dir.exists() or not split_images_dir.exists():
        print(f"⚠️  Thư mục không tồn tại")
        return None

    mask_files = list(split_masks_dir.glob("*.png"))
    mask_files = [f for f in mask_files if not f.name.endswith('_vis.png')]

    density_stats = {
        'total_images': 0,
        'images_with_planthoppers': 0,
        'total_instances': 0,
        'instances_per_image': [],
        'pixels_per_image': [],
        'class_density': {1: [], 2: [], 3: []},  # Brown, White-backed, Rice leaf miner
        'per_image_stats': []
    }

    print(f"Đang tính mật độ rầy từ {len(mask_files)} masks...")

    for mask_file in tqdm_notebook(mask_files, desc='Calculating density', leave=False) if USE_TQDM else mask_files:
        mask = cv2.imread(str(mask_file), cv2.IMREAD_GRAYSCALE)
        if mask is None:
            continue

        # Đọc ảnh để lấy kích thước
        img_file = split_images_dir / f"{mask_file.stem}.jpg"
        if not img_file.exists():
            img_file = split_images_dir / f"{mask_file.stem}.png"

        img_height, img_width = mask.shape
        image_area_pixels = img_height * img_width

        # Đếm instances cho từng class
        total_instances = 0
        image_stats = {
            'image_name': mask_file.stem,
            'image_size': (img_width, img_height),
            'instances': {},
            'pixel_counts': {},
            'density': {}
        }

        for class_id in [1, 2, 3]:
            cls_name = CLASS_MAPPING.get(class_id, f"Class_{class_id}")
            class_mask = (mask == class_id).astype(np.uint8)
            pixel_count = class_mask.sum()

            if pixel_count == 0:
                image_stats['instances'][cls_name] = 0
                image_stats['pixel_counts'][cls_name] = 0
                image_stats['density'][cls_name] = 0.0
                continue

            # Đếm số instances (contours)
            contours, _ = cv2.findContours(class_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
            num_instances = len(contours)
            total_instances += num_instances

            # Mật độ (instances / diện tích ảnh * 10000 để chuẩn hóa)
            density = (num_instances / image_area_pixels) * 10000 if image_area_pixels > 0 else 0

            image_stats['instances'][cls_name] = num_instances
            image_stats['pixel_counts'][cls_name] = pixel_count
            image_stats['density'][cls_name] = density

            density_stats['class_density'][class_id].append({
                'instances': num_instances,
                'pixels': pixel_count,
                'density': density
            })

        density_stats['total_images'] += 1
        if total_instances > 0:
            density_stats['images_with_planthoppers'] += 1
            density_stats['total_instances'] += total_instances
            density_stats['instances_per_image'].append(total_instances)
            density_stats['pixels_per_image'].append(mask[mask > 0].sum())
            density_stats['per_image_stats'].append(image_stats)

    # Tính thống kê tổng hợp
    if len(density_stats['instances_per_image']) > 0:
        density_stats['mean_instances_per_image'] = np.mean(density_stats['instances_per_image'])
        density_stats['std_instances_per_image'] = np.std(density_stats['instances_per_image'])
        density_stats['max_instances_per_image'] = np.max(density_stats['instances_per_image'])
        density_stats['min_instances_per_image'] = np.min(density_stats['instances_per_image'])

        # Thống kê theo class
        for class_id in [1, 2, 3]:
            cls_name = CLASS_MAPPING.get(class_id, f"Class_{class_id}")
            if len(density_stats['class_density'][class_id]) > 0:
                instances_list = [d['instances'] for d in density_stats['class_density'][class_id]]
                density_stats[f'{cls_name}_mean_instances'] = np.mean(instances_list)
                density_stats[f'{cls_name}_mean_density'] = np.mean([d['density'] for d in density_stats['class_density'][class_id]])

    return density_stats

# Tính mật độ rầy
print("="*70)
print("XÁC ĐỊNH MẬT ĐỘ RẦY NÂU TỪ MASKS")
print("="*70)

density_results = {}
for split in ['train', 'val', 'test']:
    print(f"\nĐang tính mật độ cho split: {split}")
    results = calculate_planthopper_density(MASKS_DIR, IMAGES_DIR, split=split)
    if results:
        density_results[split] = results

# Hiển thị kết quả
print(f"\n{'='*70}")
print("KẾT QUẢ MẬT ĐỘ RẦY NÂU")
print(f"{'='*70}")

for split, results in density_results.items():
    print(f"\nSplit: {split.upper()}")
    print(f"  Tổng số ảnh: {results['total_images']}")
    print(f"  Số ảnh có rầy: {results['images_with_planthoppers']}")
    print(f"  Tổng số cá thể (instances): {results['total_instances']}")

    if 'mean_instances_per_image' in results:
        print(f"\n  Thống kê mật độ:")
        print(f"    Trung bình: {results['mean_instances_per_image']:.2f} cá thể/ảnh")
        print(f"    Độ lệch chuẩn: {results['std_instances_per_image']:.2f}")
        print(f"    Tối đa: {results['max_instances_per_image']} cá thể/ảnh")
        print(f"    Tối thiểu: {results['min_instances_per_image']} cá thể/ảnh")

        print(f"\n  Mật độ theo loài:")
        for class_id in [1, 2, 3]:
            cls_name = CLASS_MAPPING.get(class_id, f"Class_{class_id}")
            mean_inst_key = f'{cls_name}_mean_instances'
            mean_dens_key = f'{cls_name}_mean_density'
            if mean_inst_key in results:
                print(f"    {cls_name}:")
                print(f"      Trung bình: {results[mean_inst_key]:.2f} cá thể/ảnh")
                print(f"      Mật độ chuẩn hóa: {results[mean_dens_key]:.4f}")

# Visualize phân bố mật độ
if density_results:
    fig, axes = plt.subplots(1, 2, figsize=(14, 5))

    # Phân bố số instances/ảnh
    all_instances = []
    for split, results in density_results.items():
        all_instances.extend(results.get('instances_per_image', []))

    if all_instances:
        axes[0].hist(all_instances, bins=30, edgecolor='black', alpha=0.7)
        axes[0].set_xlabel('Số lượng cá thể rầy/ảnh', fontsize=11)
        axes[0].set_ylabel('Số lượng ảnh', fontsize=11)
        axes[0].set_title('Phân bố số lượng cá thể rầy trên mỗi ảnh', fontsize=12, fontweight='bold')
        axes[0].axvline(np.mean(all_instances), color='red', linestyle='--',
                       label=f'Trung bình: {np.mean(all_instances):.2f}')
        axes[0].legend()
        axes[0].grid(True, alpha=0.3)

    # So sánh mật độ theo loài
    class_names = ['Brown\nPlanthopper', 'White-backed\nPlanthopper', 'Rice Leaf\nMiner']
    mean_instances = []
    for class_id in [1, 2, 3]:
        cls_name = CLASS_MAPPING[class_id]
        all_instances_class = []
        for split, results in density_results.items():
            for item in results['class_density'][class_id]:
                all_instances_class.append(item['instances'])
        mean_instances.append(np.mean(all_instances_class) if all_instances_class else 0)

    axes[1].bar(class_names, mean_instances, color=['#ff6b6b', '#4ecdc4', '#45b7d1'], alpha=0.7)
    axes[1].set_ylabel('Số lượng cá thể trung bình/ảnh', fontsize=11)
    axes[1].set_title('So sánh mật độ theo loài rầy', fontsize=12, fontweight='bold')
    axes[1].grid(True, alpha=0.3, axis='y')

    plt.tight_layout()
    plt.show()


## 4.3. So sánh với mô hình tham chiếu SwinT-YOLOv8-p2

So sánh kết quả của SAM-ViT với mô hình tham chiếu SwinT-YOLOv8-p2 trên các metrics: Precision, Recall, F1-score, mAP.

In [ ]:
# =============================================================================
# SO SÁNH VỚI SWINT-YOLOV8-P2
# =============================================================================

print("="*70)
print("SO SÁNH VỚI MÔ HÌNH THAM CHIẾU: SWINT-YOLOV8-P2")
print("="*70)

# Kết quả từ SwinT-YOLOv8-p2 (từ đề tài)
swint_yolov8_results = {
    'mAP@0.5': 0.847,
    'F1_score': 0.899,
    'Precision': 0.857,  # Giả định từ mAP và F1
    'Recall': 0.943,     # Giả định từ mAP và F1
    'FPS': 16.69,
    'description': 'SwinT-YOLOv8-p2: Detection model với Swin Transformer và SCConv'
}

# Kết quả từ SAM-ViT
sam_vit_results = {
    'Accuracy': test_results['accuracy'] / 100,  # Chuyển từ % sang decimal
    'F1_score': test_results['f1'],
    'Precision': test_results['precision'],
    'Recall': test_results['recall'],
    'description': 'SAM-ViT: Segmentation (SAM) + Classification (ViT)'
}

# So sánh
comparison_df = pd.DataFrame({
    'Metric': ['mAP@0.5 / Accuracy', 'Precision', 'Recall', 'F1-Score', 'FPS'],
    'SwinT-YOLOv8-p2': [
        swint_yolov8_results['mAP@0.5'],
        swint_yolov8_results['Precision'],
        swint_yolov8_results['Recall'],
        swint_yolov8_results['F1_score'],
        swint_yolov8_results['FPS']
    ],
    'SAM-ViT': [
        sam_vit_results['Accuracy'],
        sam_vit_results['Precision'],
        sam_vit_results['Recall'],
        sam_vit_results['F1_score'],
        'N/A (Classification task)'
    ]
})

print("\nBảng so sánh kết quả:")
print("="*70)
print(comparison_df.to_string(index=False))
print("="*70)

# Tính phần trăm cải thiện
improvement = {}
for metric in ['Precision', 'Recall', 'F1_score']:
    sam_vit_val = sam_vit_results[metric]
    swint_val = swint_yolov8_results[metric]
    if isinstance(sam_vit_val, (int, float)) and isinstance(swint_val, (int, float)):
        improvement[metric] = ((sam_vit_val - swint_val) / swint_val) * 100

print("\nPhần trăm cải thiện của SAM-ViT so với SwinT-YOLOv8-p2:")
for metric, pct in improvement.items():
    sign = "+" if pct >= 0 else ""
    print(f"  {metric}: {sign}{pct:.2f}%")

# Visualize comparison
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Bar chart so sánh metrics
metrics_compare = ['Precision', 'Recall', 'F1-Score']
swint_vals = [swint_yolov8_results[m] for m in ['Precision', 'Recall', 'F1_score']]
sam_vit_vals = [sam_vit_results[m] for m in ['Precision', 'Recall', 'F1_score']]

x = np.arange(len(metrics_compare))
width = 0.35

axes[0].bar(x - width/2, swint_vals, width, label='SwinT-YOLOv8-p2', alpha=0.8, color='#3498db')
axes[0].bar(x + width/2, sam_vit_vals, width, label='SAM-ViT', alpha=0.8, color='#e74c3c')
axes[0].set_ylabel('Score', fontsize=11)
axes[0].set_title('So sánh Precision, Recall, F1-Score', fontsize=12, fontweight='bold')
axes[0].set_xticks(x)
axes[0].set_xticklabels(metrics_compare)
axes[0].legend()
axes[0].grid(True, alpha=0.3, axis='y')
axes[0].set_ylim([0, 1])

# Accuracy/Detection rate comparison
models = ['SwinT-YOLOv8-p2\n(mAP@0.5)', 'SAM-ViT\n(Accuracy)']
acc_values = [swint_yolov8_results['mAP@0.5'], sam_vit_results['Accuracy']]
colors = ['#3498db', '#e74c3c']

axes[1].bar(models, acc_values, alpha=0.8, color=colors)
axes[1].set_ylabel('Score', fontsize=11)
axes[1].set_title('So sánh mAP@0.5 (Detection) vs Accuracy (Classification)',
                  fontsize=12, fontweight='bold')
axes[1].grid(True, alpha=0.3, axis='y')
axes[1].set_ylim([0, 1])

# Thêm giá trị trên bars
for i, (model, val) in enumerate(zip(models, acc_values)):
    axes[1].text(i, val + 0.02, f'{val:.3f}', ha='center', fontsize=10, fontweight='bold')

plt.tight_layout()
plt.show()

print("\n" + "="*70)
print("NHẬN XÉT SO SÁNH")
print("="*70)
print("""
📊 SwinT-YOLOv8-p2:
   - Mô hình Detection (YOLO) với Swin Transformer
   - Tốc độ inference cao (16.69 FPS)
   - Phát hiện đối tượng dựa trên bounding boxes
   - mAP@0.5 = 0.847, F1 = 0.899

📊 SAM-ViT (Đề xuất):
   - Mô hình kết hợp Segmentation (SAM) + Classification (ViT)
   - SAM tạo masks chính xác ở cấp độ pixel
   - ViT phân loại từng ROI được trích xuất từ masks
   - Ưu điểm: Phân đoạn chi tiết, xác định mật độ chính xác
   - Nhược điểm: Chậm hơn (2-stage pipeline)

🎯 Kết luận:
   - SAM-ViT phù hợp cho bài toán cần phân đoạn chi tiết và xác định mật độ
   - SwinT-YOLOv8-p2 phù hợp cho bài toán real-time detection
   - Hai phương pháp bổ sung cho nhau: SwinT-YOLOv8-p2 để detect nhanh,
     SAM-ViT để phân tích chi tiết và đánh giá mật độ
""")

# Lưu kết quả so sánh
comparison_csv = Path("/content/drive/MyDrive/ViT_checkpoints/comparison_with_swint_yolov8.csv")
comparison_df.to_csv(comparison_csv, index=False)
print(f"\n✓ Đã lưu bảng so sánh: {comparison_csv}")

print("\n" + "="*70)
print("TÓM TẮT KẾT QUẢ SAM-ViT")
print("="*70)

print(f"\nTest Set Performance:")
print(f"  Accuracy: {test_results['accuracy']:.2f}%")
print(f"  Precision: {test_results['precision']:.4f}")
print(f"  Recall: {test_results['recall']:.4f}")
print(f"  F1 Score: {test_results['f1']:.4f}")

if segmentation_results:
    print(f"\nSegmentation Quality:")
    for split, seg_results in segmentation_results.items():
        print(f"  {split.upper()}:")
        for cls_name, stats in seg_results.get('class_stats', {}).items():
            print(f"    {cls_name} - Mean IoU: {stats['mean_iou']:.4f}, Mean Dice: {stats['mean_dice']:.4f}")

if density_results:
    print(f"\nDensity Analysis:")
    for split, dens_results in density_results.items():
        if 'mean_instances_per_image' in dens_results:
            print(f"  {split.upper()}: {dens_results['mean_instances_per_image']:.2f} cá thể/ảnh")

print("\n" + "="*70)
print("ĐỀ XUẤT HƯỚNG MỞ RỘNG")
print("="*70)
print("""
1. Tối ưu hiệu suất:
   - Cải thiện tốc độ inference bằng cách tối ưu SAM pipeline
   - Sử dụng SAM nhanh hơn hoặc quantize model
   - Kết hợp SAM-ViT với SwinT-YOLOv8-p2: dùng YOLO để detect nhanh,
     SAM-ViT để phân tích chi tiết những vùng có rầy

2. Cải thiện độ chính xác:
   - Fine-tune SAM trên dữ liệu rầy nâu cụ thể
   - Ensemble nhiều ViT models với các architecture khác nhau
   - Sử dụng attention mechanisms để tập trung vào vùng rầy nhỏ

3. Xử lý mật độ cao:
   - Xử lý trường hợp rầy chồng lấp (overlapping instances)
   - Cải thiện khả năng phân tách các cá thể gần nhau
   - Sử dụng instance segmentation thay vì semantic segmentation

4. Ứng dụng thực tế:
   - Tích hợp vào hệ thống giám sát tự động trên đồng ruộng
   - Phát triển ứng dụng mobile để nông dân sử dụng
   - Kết hợp với IoT sensors và drone để giám sát quy mô lớn
   - Xây dựng hệ thống dự báo dịch hại dựa trên mật độ

5. Đánh giá mở rộng:
   - Test trên nhiều điều kiện ánh sáng, góc chụp khác nhau
   - Đánh giá hiệu quả trên các giai đoạn phát triển khác nhau của lúa
   - Cross-validation để đảm bảo tính ổn định và khả năng tổng quát hóa
""")

print("\n✓ Hoàn thành phân tích và so sánh!")

# =============================================================================
# LƯU MODEL VÀ KẾT QUẢ
# =============================================================================

print("\n" + "="*70)
print("LƯU MODEL VÀ KẾT QUẢ")
print("="*70)

# Lưu vào Google Drive
save_dir = Path("/content/drive/MyDrive/ViT_checkpoints")
save_dir.mkdir(parents=True, exist_ok=True)

# Lưu model cuối cùng
final_checkpoint = {
    'model_state_dict': model.state_dict(),
    'optimizer_state_dict': optimizer.state_dict(),
    'epoch': NUM_EPOCHS,
    'best_val_f1': best_val_f1,
    'best_epoch': best_epoch,
    'test_results': test_results,
    'class_names': class_names,
    'history': history,
    'segmentation_results': segmentation_results,
    'density_results': density_results,
    'config': {
        'num_classes': NUM_CLASSES,
        'learning_rate': LEARNING_RATE,
        'batch_size': BATCH_SIZE,
        'class_mapping': CLASS_MAPPING
    }
}

final_path = save_dir / 'vit_classifier_final.pth'
torch.save(final_checkpoint, final_path)
print(f"✓ Final model đã được lưu: {final_path}")

# Export predictions và results
predictions_df = pd.DataFrame({
    'true_label': [class_names[i] for i in test_results['labels']],
    'predicted_label': [class_names[i] for i in test_results['predictions']],
    'correct': test_results['labels'] == test_results['predictions']
})

results_csv = save_dir / 'test_results.csv'
predictions_df.to_csv(results_csv, index=False)
print(f"✓ Test results đã được lưu: {results_csv}")

# Lưu summary với đầy đủ thông tin
summary = {
    'model': 'SAM-ViT',
    'best_val_f1': best_val_f1,
    'best_epoch': best_epoch,
    'test_metrics': {
        'accuracy': test_results['accuracy'],
        'precision': test_results['precision'],
        'recall': test_results['recall'],
        'f1': test_results['f1']
    },
    'per_class_metrics': test_results['per_class'],
    'segmentation_quality': segmentation_results,
    'density_analysis': density_results,
    'comparison_with_swint_yolov8': {
        'swint_yolov8': swint_yolov8_results,
        'sam_vit': sam_vit_results,
        'improvement': improvement
    }
}

summary_json = save_dir / 'summary.json'
with open(summary_json, 'w', encoding='utf-8') as f:
    json.dump(summary, f, indent=2, ensure_ascii=False)
print(f"✓ Summary đã được lưu: {summary_json}")

# Lưu density results
if density_results:
    density_json = save_dir / 'density_results.json'
    with open(density_json, 'w', encoding='utf-8') as f:
        json.dump(density_results, f, indent=2, ensure_ascii=False, default=str)
    print(f"✓ Density results đã được lưu: {density_json}")

print(f"\n📁 Tất cả files đã được lưu vào: {save_dir}")
# Export predictions summary
predictions_df = pd.DataFrame({
    'true_label': [class_names[i] for i in test_results['labels']],
    'predicted_label': [class_names[i] for i in test_results['predictions']],
    'correct': test_results['labels'] == test_results['predictions']
})

print("\n" + "="*70)
print("TÓM TẮT PREDICTIONS")
print("="*70)
print(predictions_df.groupby('true_label')['correct'].agg(['count', 'sum', 'mean']))
print("="*70)
